In [1]:
# import required packages
import warnings
warnings.simplefilter("ignore")
import os
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from nimbus_inference.nimbus import Nimbus, prep_naming_convention
from nimbus_inference.utils import MultiplexDataset
from alpineer import io_utils
from nimbus_inference import example_dataset
from nimbus_inference.viewer_widget import NimbusViewer

In [2]:
# set up the base directory
base_dir = os.path.normpath("../../data/Nimbus")

In [19]:
# set up file paths
tiff_dir = os.path.join(base_dir, "image_data/run3")
deepcell_output_dir = os.path.join(base_dir, "segmentation")
nimbus_output_dir = os.path.join(base_dir, "nimbus_output/run3")

# Create nimbus output directory
os.makedirs(nimbus_output_dir, exist_ok=True)

# Check if paths exist
io_utils.validate_paths([base_dir, tiff_dir, deepcell_output_dir, nimbus_output_dir])

## Seperate ome tiffs into multiple tiff images, one for each channel

In [4]:
import tifffile as tiff
import os
import glob

# Directory containing your OME-TIFF files
ome_tiff_dir = tiff_dir

# Find all .ome.tif files
ome_files = glob.glob(os.path.join(ome_tiff_dir, "*.ome.tiff"))

for ome_tiff_path in ome_files:
    # Make output directory with same name as the OME-TIFF (without extension)
    base_name = os.path.splitext(os.path.basename(ome_tiff_path))[0]
    output_dir = os.path.join(ome_tiff_dir, base_name)
    os.makedirs(output_dir, exist_ok=True)

    print(f"Processing {ome_tiff_path} -> {output_dir}")

    # Load the OME-TIFF
    with tiff.TiffFile(ome_tiff_path) as tif:
        data = tif.asarray()
        ome_metadata = tif.ome_metadata

    # Handle dimensions
    if data.ndim == 5:  
        _, num_channels, _, _, _ = data.shape
    elif data.ndim == 3:  
        num_channels = data.shape[0]
    else:
        raise ValueError(f"Unexpected data shape {data.shape} in {ome_tiff_path}")

    # Save each channel
    for c in range(num_channels):
        if data.ndim == 5:
            channel_img = data[0, c, 0, :, :]  # first t, first z
        else:
            channel_img = data[c, :, :]
        
        out_path = os.path.join(output_dir, f"channel_{c+1}.tiff")
        tiff.imwrite(out_path, channel_img)
        print(f"  Saved {out_path}")


Processing ../../data/Nimbus/image_data/ID_0022111_column_4.ome.tiff -> ../../data/Nimbus/image_data/ID_0022111_column_4.ome
  Saved ../../data/Nimbus/image_data/ID_0022111_column_4.ome/channel_1.tiff
  Saved ../../data/Nimbus/image_data/ID_0022111_column_4.ome/channel_2.tiff
  Saved ../../data/Nimbus/image_data/ID_0022111_column_4.ome/channel_3.tiff
  Saved ../../data/Nimbus/image_data/ID_0022111_column_4.ome/channel_4.tiff
  Saved ../../data/Nimbus/image_data/ID_0022111_column_4.ome/channel_5.tiff
  Saved ../../data/Nimbus/image_data/ID_0022111_column_4.ome/channel_6.tiff
  Saved ../../data/Nimbus/image_data/ID_0022111_column_4.ome/channel_7.tiff
  Saved ../../data/Nimbus/image_data/ID_0022111_column_4.ome/channel_8.tiff
  Saved ../../data/Nimbus/image_data/ID_0022111_column_4.ome/channel_9.tiff
  Saved ../../data/Nimbus/image_data/ID_0022111_column_4.ome/channel_10.tiff
  Saved ../../data/Nimbus/image_data/ID_0022111_column_4.ome/channel_11.tiff
  Saved ../../data/Nimbus/image_data/

In [5]:
# Path to your parent directory that contains the folders
parent_dir = tiff_dir

for folder in os.listdir(parent_dir):
    old_path = os.path.join(parent_dir, folder)
    if os.path.isdir(old_path) and folder.endswith(".ome"):
        new_name = folder[:-4]  # remove the ".ome"
        new_path = os.path.join(parent_dir, new_name)
        os.rename(old_path, new_path)
        print(f"Renamed: {old_path} -> {new_path}")


Renamed: ../../data/Nimbus/image_data/ID_56777_1_1_core_4.ome -> ../../data/Nimbus/image_data/ID_56777_1_1_core_4
Renamed: ../../data/Nimbus/image_data/ID_0022111_column_4.ome -> ../../data/Nimbus/image_data/ID_0022111_column_4
Renamed: ../../data/Nimbus/image_data/ID_56764.ome -> ../../data/Nimbus/image_data/ID_56764
Renamed: ../../data/Nimbus/image_data/ID_56777_1_2.ome -> ../../data/Nimbus/image_data/ID_56777_1_2
Renamed: ../../data/Nimbus/image_data/ID_0022111_column_3.ome -> ../../data/Nimbus/image_data/ID_0022111_column_3
Renamed: ../../data/Nimbus/image_data/ID_0022110_column_4.ome -> ../../data/Nimbus/image_data/ID_0022110_column_4
Renamed: ../../data/Nimbus/image_data/ID_0022110_column_3.ome -> ../../data/Nimbus/image_data/ID_0022110_column_3
Renamed: ../../data/Nimbus/image_data/ID_56777_2.ome -> ../../data/Nimbus/image_data/ID_56777_2
Renamed: ../../data/Nimbus/image_data/ID_0022111_column_1.ome -> ../../data/Nimbus/image_data/ID_0022111_column_1
Renamed: ../../data/Nimbus/i

### Rename channel images with their names

In [21]:
parent_dir = tiff_dir

markers = ['DAPI', 'FoxP3', 'aSMA', 'CD4', 'CD8', 'CD31', 'CD11c', 'IFNG', 'Pan-Cytokeratin', 'CD68', 'CD20', 'CD66b', 'TNFa', 'CD45RO', 'CD14', 'CD11b', 'Vimentin', 'CD163', 'PDGFRA', 'CD45', 'CCR7', 'IL10', 'CD38', 'CD69', 'Podoplanin', 'PNAd', 'ECP', 'MPO', 'MIP-3']

# Loop over all sample folders
for sample_folder in os.listdir(parent_dir):
    sample_path = os.path.join(parent_dir, sample_folder)
    if not os.path.isdir(sample_path):
        continue  # skip files

    # Rename each channel file
    for idx, marker in enumerate(markers, start=1):
        old_file = os.path.join(sample_path, f"channel_{idx}.tiff")
        new_file = os.path.join(sample_path, f"{marker}.tiff")
        if os.path.exists(old_file):
            os.rename(old_file, new_file)
            print(f"Renamed: {old_file} -> {new_file}")
        else:
            print(f"Warning: {old_file} does not exist")

Renamed: ../../data/Nimbus/image_data/run3/ID_56777_1_2/channel_1.tiff -> ../../data/Nimbus/image_data/run3/ID_56777_1_2/DAPI.tiff
Renamed: ../../data/Nimbus/image_data/run3/ID_56777_1_2/channel_2.tiff -> ../../data/Nimbus/image_data/run3/ID_56777_1_2/FoxP3.tiff
Renamed: ../../data/Nimbus/image_data/run3/ID_56777_1_2/channel_3.tiff -> ../../data/Nimbus/image_data/run3/ID_56777_1_2/aSMA.tiff
Renamed: ../../data/Nimbus/image_data/run3/ID_56777_1_2/channel_4.tiff -> ../../data/Nimbus/image_data/run3/ID_56777_1_2/CD4.tiff
Renamed: ../../data/Nimbus/image_data/run3/ID_56777_1_2/channel_5.tiff -> ../../data/Nimbus/image_data/run3/ID_56777_1_2/CD8.tiff
Renamed: ../../data/Nimbus/image_data/run3/ID_56777_1_2/channel_6.tiff -> ../../data/Nimbus/image_data/run3/ID_56777_1_2/CD31.tiff
Renamed: ../../data/Nimbus/image_data/run3/ID_56777_1_2/channel_7.tiff -> ../../data/Nimbus/image_data/run3/ID_56777_1_2/CD11c.tiff
Renamed: ../../data/Nimbus/image_data/run3/ID_56777_1_2/channel_8.tiff -> ../../dat

## Set up input paths

In [22]:
# define the channels to include
include_channels = ['DAPI', 'FoxP3', 'aSMA', 'CD4', 'CD8', 'CD31', 'CD11c', 'IFNG', 'Pan-Cytokeratin', 'CD68', 'CD20', 'CD66b', 'TNFa', 'CD45RO', 'CD14', 'CD11b', 'Vimentin', 'CD163', 'PDGFRA', 'CD45', 'CCR7', 'IL10', 'CD38', 'CD69', 'Podoplanin', 'PNAd', 'ECP', 'MPO', 'MIP-3']

# either get all fovs in the folder...
fov_names = os.listdir(tiff_dir)
# ... or optionally, select a specific set of fovs manually
# fovs = ["fov0", "fov1"]

# make sure to filter paths out that don't lead to FoVs, e.g. .DS_Store files.
fov_names = [fov_name for fov_name in fov_names if not fov_name.startswith(".")] 

# construct paths for fovs
fov_paths = [os.path.join(tiff_dir, fov_name) for fov_name in fov_names]

In [23]:
# Prepare segmentation naming convention that maps a fov_path to the according segmentation label map
segmentation_naming_convention = prep_naming_convention(deepcell_output_dir)

# test segmentation_naming_convention
if os.path.exists(segmentation_naming_convention(fov_paths[0])):
    print("Segmentation data exists for fov 0 and naming convention is correct")
else:
    print("Segmentation data does not exist for fov 0 or naming convention is incorrect")

Segmentation data exists for fov 0 and naming convention is correct


In [31]:
dataset = MultiplexDataset(
    fov_paths=fov_paths[3:],
    suffix=".tiff", # or .png, .jpg, .jpeg, .tif or .ome.tiff
    include_channels=include_channels,
    segmentation_naming_convention=segmentation_naming_convention,
    output_dir=	nimbus_output_dir,
)

All inputs are valid


In [29]:
fov_paths

['../../data/Nimbus/image_data/run3/ID_56777_1_2',
 '../../data/Nimbus/image_data/run3/ID_56764',
 '../../data/Nimbus/image_data/run3/ID_56777',
 '../../data/Nimbus/image_data/run3/ID_56777_1_1_core_1',
 '../../data/Nimbus/image_data/run3/ID_56777_1_1_core_4',
 '../../data/Nimbus/image_data/run3/ID_56777_1_1_core_3',
 '../../data/Nimbus/image_data/run3/ID_56777_2',
 '../../data/Nimbus/image_data/run3/ID_56777_1_1_core_2']

## Initialize model and make predictions

In [32]:
nimbus = Nimbus(
    dataset=dataset,
    save_predictions=True,
    batch_size=4,
    test_time_aug=False,
    input_shape=[1024,1024],
    device="auto",
    output_dir=nimbus_output_dir,
)

# check if all inputs are valid
nimbus.check_inputs()

Checking for updated model checkpoints on HuggingFace Hub...
Using existing checkpoint: /scratch/lyarab/micromamba/envs/nimbus_env_new/lib/python3.11/site-packages/nimbus_inference/assets/V1.pt
Loaded weights from /scratch/lyarab/micromamba/envs/nimbus_env_new/lib/python3.11/site-packages/nimbus_inference/assets/V1.pt
All inputs are valid.


In [33]:
dataset.prepare_normalization_dict(
    quantile=0.999,
    n_subset=50,
    clip_values=(0, 2),
    multiprocessing=True,
    overwrite=True
)

Iterate over fovs...


In [34]:
cell_table = nimbus.predict_fovs()

Available GPUs:  0
Predictions will be saved in ../../data/Nimbus/nimbus_output/run3
Iterating through fovs will take a while...
Predicting ../../data/Nimbus/image_data/run3/ID_56777_1_1_core_1...


  0%|          | 0/29 [00:00<?, ?it/s]

Checking for updated model checkpoints on HuggingFace Hub...
Using existing checkpoint: /scratch/lyarab/micromamba/envs/nimbus_env_new/lib/python3.11/site-packages/nimbus_inference/assets/V1.pt
Loaded weights from /scratch/lyarab/micromamba/envs/nimbus_env_new/lib/python3.11/site-packages/nimbus_inference/assets/V1.pt


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [28]:
viewer = NimbusViewer(dataset=dataset, output_dir=nimbus_output_dir)
viewer.display()